In [1]:
# This is a workbook used to analyse the movement of users through data that was collected from April 2007 to April 2012
# This was done by dividing the total area into zones and then calculating the number of users in each zone every hour
# It is also important to state the number of users in each zone per hour was calculated differently for the weekdays and weekends
# Import all the required packages 

import skmob
import geopandas as gpd
from skmob.preprocessing import detection, clustering, compression
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from skmob.tessellation import tilers

In [2]:
# Read the csv file
# tdf = skmob.TrajDataFrame.from_file('newGeolife.csv')
df = pd.read_csv('newGeolife.csv')

In [3]:
df.dtypes

Unnamed: 0      int64
lat           float64
lng           float64
datetime       object
uid            object
dtype: object

In [4]:
# Remove the unnamed column
df.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
# use skmob to read the loaded dataframe
tdf = skmob.TrajDataFrame(df, latitude='lat', longitude='lng', datetime = 'datetime')

In [6]:
# stdf = detection.stops(tdf, stop_radius_factor=0.5, minutes_for_a_stop=20.0, spatial_radius_km=0.2, leaving_time=True)

In [6]:
# Remove points that falls outside the boundary of the study-site
tdf=tdf[(tdf['lat']<40.35) & (tdf['lat']>39.7)]
tdf=tdf[(tdf['lng']<116.8) & (tdf['lng']>116)]

In [7]:
tdf.dtypes

lat                float64
lng                float64
datetime    datetime64[ns]
uid                 object
dtype: object

In [8]:
# Split the datetime column into two columns named date and time
tdf['date'] = tdf['datetime'].dt.strftime('%Y-%m-%d')
tdf['time'] = tdf['datetime'].dt.strftime('%H:%M:%S')

In [9]:
tdf.dtypes

lat                float64
lng                float64
datetime    datetime64[ns]
uid                 object
date                object
time                object
dtype: object

In [10]:
tdf.head()

,lat,lng,datetime,uid,date,time
0,39.984683,116.318450,2008-10-23 02:53:10,0,2008-10-23,02:53:10
1,39.984686,116.318417,2008-10-23 02:53:15,0,2008-10-23,02:53:15
2,39.984688,116.318385,2008-10-23 02:53:20,0,2008-10-23,02:53:20
3,39.984655,116.318263,2008-10-23 02:53:25,0,2008-10-23,02:53:25
4,39.984611,116.318026,2008-10-23 02:53:30,0,2008-10-23,02:53:30


In [11]:
# Tessellating the study-site
# Tessellation refers to the division of the points on the studies into different zones
tessellation = tilers.tiler.get("squared", base_shape="Beijing, China", meters=15000)
mtdf = tdf.mapping(tessellation)
# tile_ID column is created specifying the zone each points fall into 
mtdf.head()

/home/oluleyepeters/miniconda3/envs/env3/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/oluleyepeters/miniconda3/envs/env3/lib/python3.8/site-packages/skmob/core/trajectorydataframe.py:322: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  tile_ids = gpd.sjoin(gdf, tessellation, how=how, op='within')[[constants.TILE_ID]]


,lat,lng,datetime,uid,date,time,tile_ID
0,39.984683,116.318450,2008-10-23 02:53:10,0,2008-10-23,02:53:10,63
1,39.984686,116.318417,2008-10-23 02:53:15,0,2008-10-23,02:53:15,63
2,39.984688,116.318385,2008-10-23 02:53:20,0,2008-10-23,02:53:20,63
3,39.984655,116.318263,2008-10-23 02:53:25,0,2008-10-23,02:53:25,63
4,39.984611,116.318026,2008-10-23 02:53:30,0,2008-10-23,02:53:30,63


In [12]:
# ctdf = compression.compress(mtdf, spatial_radius_km=0.1)

In [14]:
# ctdf['year'] = pd.DatetimeIndex(ctdf['datetime']).year
# ctdf['month'] = pd.DatetimeIndex(ctdf['datetime']).month
# ctdf['day'] = pd.DatetimeIndex(ctdf['datetime']).day
# ctdf['hour'] = pd.to_datetime(ctdf['time'], format='%H:%M:%S').dt.hour
# ctdf['newTime'] = pd.to_datetime(ctdf[['year','month','day','hour']])

In [12]:
# Create a new column of year, month and day
# A new column named time_hour was created. This column depended on the already existing datetime column
# A point with a datetime value of 2008-10-23 02:53:10 will have a hour value of 02:00:00
# This makes it easier to group and calculate the number of users per hour

mtdf['year'] = pd.DatetimeIndex(mtdf['datetime']).year
mtdf['month'] = pd.DatetimeIndex(mtdf['datetime']).month
mtdf['day'] = pd.DatetimeIndex(mtdf['datetime']).day
mtdf['hour'] = pd.to_datetime(mtdf['time'], format='%H:%M:%S').dt.hour
mtdf['newTime'] = pd.to_datetime(mtdf[['year','month','day','hour']])
mtdf['time_hour'] = mtdf['newTime'].dt.time

In [12]:
mtdf.head()

,lat,lng,datetime,uid,date,time,tile_ID,year,month,day,hour,newTime,time_hour
0,39.984683,116.318450,2008-10-23 02:53:10,0,2008-10-23,02:53:10,63,2008,10,23,2,2008-10-23 02:00:00,02:00:00
1,39.984686,116.318417,2008-10-23 02:53:15,0,2008-10-23,02:53:15,63,2008,10,23,2,2008-10-23 02:00:00,02:00:00
2,39.984688,116.318385,2008-10-23 02:53:20,0,2008-10-23,02:53:20,63,2008,10,23,2,2008-10-23 02:00:00,02:00:00
3,39.984655,116.318263,2008-10-23 02:53:25,0,2008-10-23,02:53:25,63,2008,10,23,2,2008-10-23 02:00:00,02:00:00
4,39.984611,116.318026,2008-10-23 02:53:30,0,2008-10-23,02:53:30,63,2008,10,23,2,2008-10-23 02:00:00,02:00:00


In [13]:
# The analysis is to check the number of users in each zone per hour.
# Every user that appears more than once in each zone is dropped.
# Results in a user appearing in each zone not more than once
mtdf = mtdf.drop_duplicates(
  subset = ['tile_ID', 'uid', 'newTime'],
  keep = 'last').reset_index(drop = True)

In [22]:
mtdf.head()
mtdf.dtypes

lat                 float64
lng                 float64
datetime     datetime64[ns]
uid                  object
date                 object
time                 object
tile_ID              object
year                  int64
month                 int64
day                   int64
hour                  int64
newTime      datetime64[ns]
time_hour            object
dtype: object

In [39]:
# The analysis compares the maximum number of users obtainable during the weekdays and weekends
# Thus the day_of_the week value was created for each point
mtdf['day_of_week'] = mtdf['datetime'].dt.dayofweek

In [ ]:
mtdf.head()
# export the processed dataset as a new dateset
mtdf.to_csv('processed.csv')
# The exported file was opened in excel, and the numbers of a new column named day_name was created
# This column categorised the day_of_week for each point as either a weekday or weekend

In [61]:
# Read processed csv file
df = pd.read_csv('processed.csv')
# Exporting a dataframe and reading it again usually lead to the conversion of the datetime data type to objects
df.dtypes

Unnamed: 0       int64
lat            float64
lng            float64
datetime        object
uid              int64
date            object
time            object
tile_ID          int64
year             int64
month            int64
day              int64
hour             int64
newTime         object
time_hour       object
day_of_week      int64
day_name        object
value            int64
dtype: object

In [62]:
# Convert datetime columns from object type to datetime type
df['datetime'] = pd.to_datetime(df['datetime'])
df['newTime'] = pd.to_datetime(df['newTime'])
df['time'] = pd.to_datetime(df['time'])

In [63]:
df.dtypes

Unnamed: 0              int64
lat                   float64
lng                   float64
datetime       datetime64[ns]
uid                     int64
date                   object
time           datetime64[ns]
tile_ID                 int64
year                    int64
month                   int64
day                     int64
hour                    int64
newTime        datetime64[ns]
time_hour              object
day_of_week             int64
day_name               object
value                   int64
dtype: object

In [64]:
# Extract, create and export seperate dateframes for both weekend and weekday points

weekend = df[df['day_name'] == 'Weekend']
weekend.to_csv('weekend.csv')
weekday = df[df['day_name'] == 'Weekday']
weekday.to_csv('weekday.csv')

In [65]:
weekend = pd.read_csv('weekend.csv')
weekday = pd.read_csv('weekday.csv')
# convert datetime data that was converted to object during csv file creation back to datetime data type
weekday['datetime'] = pd.to_datetime(weekday['datetime'])
weekday['newTime'] = pd.to_datetime(weekday['newTime'])
weekday['time'] = pd.to_datetime(weekday['time'])
weekend['datetime'] = pd.to_datetime(weekend['datetime'])
weekend['newTime'] = pd.to_datetime(weekend['newTime'])
weekend['time'] = pd.to_datetime(weekend['time'])

In [66]:
weekday.head()
# Drop unecessary columns
weekday.drop('Unnamed: 0', axis=1, inplace=True)
weekday.drop('Unnamed: 0.1', axis=1, inplace=True)

In [67]:
weekend.head()
# Drop Unecessary column
weekend.drop('Unnamed: 0', axis=1, inplace=True)
weekend.drop('Unnamed: 0.1', axis=1, inplace=True)

In [68]:
# Create a list for of the tile_ID(zone created from tessellation) for both weekend and weekday file
weekendtile_List = [tile for tile in weekend['tile_ID']]
weekendtile_List = set(weekendtile_List)
weekendtile_List = list(weekendtile_List)
weekendtile_List 

weekdaytile_List = [tile for tile in weekday['tile_ID']]
weekdaytile_List = set(weekdaytile_List)
weekdaytile_List = list(weekdaytile_List)
weekendtile_List 

[36,
 37,
 38,
 39,
 40,
 41,
 42,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 120,
 121,
 122,
 123,
 124,
 125]

In [133]:
import copy
# create column headings for a dataframe to be created
column_names = ['time_hour']
# extend the column headings with the list of tile_ID created initially
column_names.extend(weekdaytile_List)
# Create a dataframe with this column headings
weekdaydf = pd.DataFrame(columns = column_names)
# get all the unique values of time_hour present in the weekday dataframe and add as a column in the new dataframe
time_hour = pd.DataFrame(weekday.time_hour.unique())
weekdaydf['time_hour'] = copy.deepcopy(time_hour[0])
# Loop thrugh the weekdaytile_List and use it to fill the new dataframe with the maximum number of users that could be gotten in each zone per hour during the week.
for index in weekdaytile_List:
    newDf = weekday[weekday['tile_ID'] == index]
    newDf = newDf.groupby(['time_hour','uid'])['value'].nunique()
    newDf = newDf.reset_index()
    newDf = newDf.groupby(['time_hour'])['value'].sum()
    newFrame = pd.DataFrame(newDf)
    newFrame = newFrame.reset_index()
    for i, row in newFrame.iterrows():
        weekdaydf.loc[row.time_hour, index] = row.value

weekdaydf = weekdaydf.iloc[24:]
# replace the Nan filled cells with 0
weekdaydf.drop('time_hour',  axis=1, inplace=True)
weekdaydf =  weekdaydf.rename(columns= {'index': 'Time'}).fillna(0).sort_index().transpose()
weekdaydf

,00:00:00,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,07:00:00,08:00:00,09:00:00,...,14:00:00,15:00:00,16:00:00,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00
36,1,1,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,1
37,2,2,0,0,0,0,0,1,1,1,...,1,0,0,0,0,0,0,1,1,0
38,1,1,2,3,3,3,3,4,2,0,...,0,0,0,0,0,0,0,0,0,1
39,3,3,3,5,5,4,2,1,1,4,...,1,0,0,1,0,0,0,0,0,3
40,3,1,3,6,5,6,5,5,4,4,...,0,0,0,0,0,0,0,0,0,1
41,0,0,0,1,0,1,1,2,3,1,...,0,0,0,0,0,0,0,0,0,0
42,5,3,2,7,4,3,6,6,4,4,...,2,0,0,1,0,0,0,0,0,1
47,2,2,2,1,4,1,1,4,3,4,...,2,0,0,0,0,0,0,0,0,2
48,3,9,6,4,11,9,3,7,8,5,...,3,1,1,0,0,0,0,1,4,4
49,9,12,12,10,15,13,10,13,18,12,...,5,3,3,0,0,0,0,0,8,5


In [132]:
import copy
# create column headings for a dataframe to be created
column_names = ['time_hour']
# extend the column headings with the list of tile_ID created initially
column_names.extend(weekendtile_List)
# Create a dataframe with this column headings
weekenddf = pd.DataFrame(columns = column_names)
# get all the unique values of time_hour present in the weekday dataframe and add as a column in the new dataframe
time_hour = pd.DataFrame(weekend.time_hour.unique())
weekenddf['time_hour'] = copy.deepcopy(time_hour[0])
# Loop thrugh the weekdaytile_List and use it to fill the new dataframe with the maximum number of users that could be gotten in each zone per hour during the week.
for index in weekendtile_List:
    newDf = weekend[weekend['tile_ID'] == index]
    newDf = newDf.groupby(['time_hour','uid'])['value'].nunique()
    newDf = newDf.reset_index()
    newDf = newDf.groupby(['time_hour'])['value'].sum()
    newFrame = pd.DataFrame(newDf)
    newFrame = newFrame.reset_index()
    for i, row in newFrame.iterrows():
        weekenddf.loc[row.time_hour, index] = row.value

weekenddf = weekenddf.iloc[24:]
# replace the Nan filled cells with 0
weekenddf.drop('time_hour',  axis=1, inplace=True)
weekenddf =  weekenddf.rename(columns= {'index': 'Time'}).fillna(0).sort_index().transpose()
weekenddf

,00:00:00,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,07:00:00,08:00:00,09:00:00,...,14:00:00,15:00:00,16:00:00,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00
36,0,1,0,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
37,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
38,0,1,1,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
39,0,1,1,1,1,2,2,1,1,2,...,0,0,0,0,0,0,0,0,0,0
40,0,0,1,3,3,3,3,3,2,2,...,1,1,0,0,0,0,0,0,0,0
41,0,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
42,2,2,3,2,2,2,2,2,7,2,...,1,2,1,1,0,0,0,0,1,2
47,0,2,1,2,1,2,0,1,2,1,...,1,0,0,0,0,0,0,0,2,0
48,3,3,1,3,3,6,1,1,6,4,...,0,0,0,0,0,0,0,0,7,1
49,3,4,2,4,3,4,4,9,10,9,...,2,0,0,0,0,0,1,0,8,4
